In [2]:
import torch,sys,pickle,os,time
import numpy as np
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv
from tqdm import tqdm

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


data_list = range(2101,2400)
for data_number in data_list:
    if os.path.exists('../data/both57m_rigged/%d.pkl' % data_number):
        start_time = time.time()
        print('Processing data %d' % data_number)
        xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
        env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)

        with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
            data = pickle.load(f)

        ZERO_ROOT = True
        video = []
        tick = 0
        seq_len = data['length']

        qpos_rig_data = data['qpos']
        ls2le_array = np.zeros((seq_len,3))
        le2lw_array = np.zeros((seq_len,3))
        rs2re_array = np.zeros((seq_len,3))
        re2rw_array = np.zeros((seq_len,3))
        lw_rot_array = np.zeros((seq_len,3,3))
        rw_rot_array = np.zeros((seq_len,3,3))
        lh_array = np.zeros((seq_len,6))
        rh_array = np.zeros((seq_len,6))
        seq_array = np.zeros((2,seq_len,3))

        while tick < seq_len:
            q_rig = qpos_rig_data[tick,:]

            if ZERO_ROOT:
                q_rig[3:7] = 0
            
            env.forward(q=q_rig,INCREASE_TICK=True)
            ls = env.get_p_body('left_shoulder')
            le = env.get_p_body('left_elbow')
            lw = env.get_p_body('left_wrist')
            rs = env.get_p_body('right_shoulder')
            re = env.get_p_body('right_elbow')
            rw = env.get_p_body('right_wrist')

            ls2le_array[tick,:] = np_uv(le-ls)
            le2lw_array[tick,:] = np_uv(lw-le)
            rs2re_array[tick,:] = np_uv(re-rs)
            re2rw_array[tick,:] = np_uv(rw-re)
            lw_rot_array[tick,:] = env.get_R_body('left_wrist')
            rw_rot_array[tick,:] = env.get_R_body('right_wrist')
            tick = tick + 1

        vis_tick = 0
        env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                        viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        recon_tick = 0

        env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                        viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        BODY_NAMES = ['left_elbow','left_wrist','right_elbow','right_wrist']
        JOINT_NAMES = ['l_shoulder1', 'l_shoulder2', 'l_shoulder3', 'l_elbow', 'l_wrist1', 'l_wrist2', 'l_wrist3',
                    'r_shoulder1', 'r_shoulder2', 'r_shoulder3', 'r_elbow', 'r_wrist1', 'r_wrist2', 'r_wrist3']

        hand_indices = list(range(27,49)) + list(range(54,76))
        video = []
        wrist_dist_list = []
        smplh_lw_pos_list = []
        smplh_rw_pos_list = []
        q_smplh_up_list = []

        ls = env.get_p_body('left_shoulder')
        le = env.get_p_body('left_elbow')
        lw = env.get_p_body('left_wrist')
        rs = env.get_p_body('right_shoulder')
        re = env.get_p_body('right_elbow')
        rw = env.get_p_body('right_wrist')

        len_ls2le = np.linalg.norm(le-ls)
        len_le2lw = np.linalg.norm(lw-le)
        len_rs2re = np.linalg.norm(re-rs)
        len_re2rw = np.linalg.norm(rw-re)

        while recon_tick < seq_len:
            p_target = {}
            p_target['left_elbow'] = ls + len_ls2le * ls2le_array[recon_tick,:]
            p_target['left_wrist'] = p_target['left_elbow'] + len_le2lw * le2lw_array[recon_tick,:]
            p_target['right_elbow'] = rs + len_rs2re * rs2re_array[recon_tick,]
            p_target['right_wrist'] = p_target['right_elbow'] + len_re2rw * re2rw_array[recon_tick,:]
            
            idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
            q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
            joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
            joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

            q_body = env.get_qpos_joints(JOINT_NAMES)
            body_diff = 1e3
            prev_q_pos = env.data.qpos.copy()

            while body_diff > 1e-3:
                for ik_tick in range(1):
                    J_list,ik_err_list = [],[]
                    for ik_body_name in BODY_NAMES:
                        ik_p_trgt = p_target[ik_body_name]
                        ik_R_trgt = None
                        IK_P = True
                        IK_R = False
                        J,ik_err = env.get_ik_ingredients(
                            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                            IK_P=IK_P,IK_R=IK_R)
                        J_list.append(J)
                        ik_err_list.append(ik_err)
                    J_stack      = np.vstack(J_list)
                    ik_err_stack = np.hstack(ik_err_list)
                    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                    q_body = q_body + dq[joint_idxs_jac_body]

                    q_body = np.clip(q_body,q_mins_body,q_maxs_body)
                    env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

                    q_env = env.data.qpos.copy()
                    body_diff = np.linalg.norm(q_env - prev_q_pos)
                    prev_q_pos=q_env.copy()

            q_rig = qpos_rig_data[recon_tick,:]
            q_env[hand_indices] = q_rig[hand_indices]
            env.forward(q=q_env,INCREASE_TICK=True)
            q_smplh_up_list.append(q_env)

            left_wrist_pos = env.get_p_body('left_wrist')
            right_wrist_pos = env.get_p_body('right_wrist')
            smplh_lw_pos_list.append(left_wrist_pos)
            smplh_rw_pos_list.append(right_wrist_pos)
            wrist_dist = np.linalg.norm(left_wrist_pos - right_wrist_pos)
            wrist_dist_list.append(wrist_dist)
            wrist_pos = np.vstack((left_wrist_pos,right_wrist_pos))
            recon_tick += 1

        q_smplh_up_array = np.array(q_smplh_up_list)
        env.close_viewer()

        xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
        env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
        q_init = env.data.qpos.copy()
        q_init[18] = -np.pi/2
        env.forward(q=q_init)

        robot_right_shoulder_pos = env.get_p_joint('robot2/joint1')
        robot_right_wrist_pos = env.get_p_joint('robot2/joint6')
        robot_left_shoulder_pos = env.get_p_joint('robot1/joint1')

        robot_arm_length = np.linalg.norm(robot_right_shoulder_pos-robot_right_wrist_pos)

        env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        BODY_NAMES = ['robot1/end_link','robot2/end_link']
        JOINT_NAMES = ['robot1/joint1','robot1/joint2','robot1/joint3','robot1/joint4','robot1/joint5','robot1/joint6',
                    'robot2/joint1','robot2/joint2','robot2/joint3','robot2/joint4','robot2/joint5','robot2/joint6']

        ls = env.get_p_body('robot1/link2')
        le = env.get_p_body('robot1/link4')
        lw = env.get_p_body('robot1/link6')
        rs = env.get_p_body('robot2/link2')
        re = env.get_p_body('robot2/link4')
        rw = env.get_p_body('robot2/link6')

        len_ls2le = np.linalg.norm(le-ls)
        len_le2lw = np.linalg.norm(lw-le)
        len_rs2re = np.linalg.norm(re-rs)
        len_re2rw = np.linalg.norm(rw-re)

        q_env_array = np.zeros((seq_len,env.model.nq))

        for tick in tqdm(range(seq_len), desc="Processing"):
            p_target = {}
            delta_lw = len_ls2le * ls2le_array[tick,:] + len_le2lw * le2lw_array[tick,:]
            p_target['robot1/end_link'] = ls + delta_lw
            delta_rw = len_rs2re * rs2re_array[tick,:] + len_re2rw * re2rw_array[tick,:]
            p_target['robot2/end_link'] = rs + delta_rw
            
            R_target = {}
            R_target['robot1/end_link'] = lw_rot_array[tick,:] @ rpy2r(np.radians([0,0,90]))
            R_target['robot2/end_link'] = rw_rot_array[tick,:] @ rpy2r(np.radians([0,0,-90]))

            idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
            q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
            joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
            joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

            q_body = env.get_qpos_joints(JOINT_NAMES)
            body_diff = 1e3
            prev_q_pos = env.data.qpos.copy()

            while body_diff > 1e-3:
                for ik_tick in range(1):
                    J_list,ik_err_list = [],[]
                    for ik_body_name in BODY_NAMES:
                        ik_p_trgt = p_target[ik_body_name]
                        ik_R_trgt = R_target[ik_body_name]
                        IK_P = True
                        IK_R = True
                        J,ik_err = env.get_ik_ingredients(
                            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                            IK_P=IK_P,IK_R=IK_R)
                        J_list.append(J)
                        ik_err_list.append(ik_err)
                    J_stack      = np.vstack(J_list)
                    ik_err_stack = np.hstack(ik_err_list)
                    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                    q_body = q_body + dq[joint_idxs_jac_body]

                    q_body = np.clip(q_body,q_mins_body,q_maxs_body)
                    env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

                    q_env = env.data.qpos.copy()
                    body_diff = np.linalg.norm(q_env - prev_q_pos)
                    prev_q_pos=q_env.copy()

                curr_time = time.time()
                if curr_time - start_time > 15:
                    break

            q_env_array[tick,:] = q_env
            curr_time = time.time()
            if curr_time - start_time > 15:
                break

        seq_array[0,:,:] = np.array(smplh_lw_pos_list)
        seq_array[1,:,:] = np.array(smplh_rw_pos_list)
        np.save('data/smplh_sequence/%d.npy' % data_number, seq_array)
        np.save('data/baseline_data/%d.npy' % data_number,q_env_array)
        env.close_viewer()


Processing data 2101
Processing data 2102
Processing data 2103
Processing data 2104
Processing data 2105
Processing data 2106
Processing data 2107
Processing data 2108
Processing data 2109
Processing data 2110
Processing data 2111
Processing data 2112
Processing data 2113
Processing data 2114
Processing data 2115
Processing data 2116
Processing data 2117
Processing data 2118
Processing data 2119
Processing data 2120
Processing data 2121
Processing data 2122
Processing data 2123
Processing data 2124
Processing data 2125
Processing data 2126
Processing data 2127
Processing data 2128
Processing data 2129
Processing data 2130
Processing data 2131
Processing data 2132
Processing data 2133
Processing data 2134
Processing data 2135
Processing data 2136
Processing data 2137
Processing data 2138
Processing data 2139
Processing data 2140
Processing data 2141
Processing data 2142
Processing data 2143
Processing data 2144
Processing data 2145
Processing data 2146
Processing data 2147
Processing da

In [5]:
import torch,sys,pickle,os,time
import numpy as np
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv
from scipy.spatial import procrustes

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)

q_init = env.data.qpos.copy()
q_init[18] = -np.pi/2
env.forward(q=q_init)

robot_right_shoulder_pos = env.get_p_joint('robot2/joint1')
robot_right_wrist_pos = env.get_p_joint('robot2/joint6')
robot_left_shoulder_pos = env.get_p_joint('robot1/joint1')

robot_arm_length = np.linalg.norm(robot_right_shoulder_pos-robot_right_wrist_pos)

data_list = range(618)
for data_number in data_list:
    if os.path.exists('data/baseline_data/%d.npy' % data_number):
        env.init_viewer(viewer_title='Dual-Arm',viewer_width=800,viewer_height=800,
                        viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        q_robot_full = np.load('data/baseline_data/%d.npy' % data_number)
        plot_tick = 0

        robot_left_wrist_list = []
        robot_right_wrist_list = []

        while plot_tick < q_robot_full.shape[0]:
            q_robot = q_robot_full[plot_tick,:]
            env.forward(q=q_robot,INCREASE_TICK=True)

            robot_left_wrist_pos = env.get_p_body('robot1/end_link')
            robot_right_wrist_pos = env.get_p_body('robot2/end_link')

            robot_left_wrist_list.append(robot_left_wrist_pos)
            robot_right_wrist_list.append(robot_right_wrist_pos)

            plot_tick += 1

        robot_left_wrist_array = np.array(robot_left_wrist_list)
        robot_right_wrist_array = np.array(robot_right_wrist_list)

        smplh_wrist_array = np.load('data/smplh_sequence/%d.npy' % data_number)
        smplh_left_wrist_array = smplh_wrist_array[0,:,:]
        smplh_right_wrist_array = smplh_wrist_array[1,:,:]

        print("Processing data %d" % data_number)
        la, lb, ld = procrustes(smplh_left_wrist_array,robot_left_wrist_array)
        ra, rb, rd = procrustes(smplh_right_wrist_array,robot_right_wrist_array)

        if ld > 0.3 or rd > 0.3:
            scale_data = {
                'data_number': data_number,
                'left': ld,
                'right': rd,
                'SCALE_LEFT': True if ld > 0.3 else False,
                'SCALE_RIGHT': True if rd > 0.3 else False
            }

            with open('data/scale_data/%d.pkl' % data_number, 'wb') as f:
                pickle.dump(scale_data, f)

            print("Left:", ld)
            print("Right:", rd)

        env.close_viewer()

Processing data 0
Left: 0.0033450713047809572
Right: 0.4902971845342796
Processing data 1
Processing data 2
Left: 0.9217432052217023
Right: 0.39374873181138376
Processing data 3
Left: 0.8870322897462326
Right: 0.11282979009071223
Processing data 4
Left: 0.4306456338950102
Right: 0.13300666820764384
Processing data 5
Processing data 6
Processing data 7
Left: 0.9012671240432168
Right: 0.3135874107087109
Processing data 8
Processing data 9
Processing data 10
Processing data 11
Processing data 12
Processing data 13
Left: 0.4083303982640286
Right: 0.45814288200669473
Processing data 14
Processing data 15
Processing data 16
Left: 0.15298412884335355
Right: 0.36109189377473594
Processing data 17
Left: 0.24150579056584257
Right: 0.48117571280418325
Processing data 18
Left: 0.6546360731121528
Right: 0.5887042779954829
Processing data 19
Left: 0.8278133689781988
Right: 0.5284922064148052
Processing data 20
Processing data 21
Processing data 22
Processing data 23
Processing data 24
Processing dat

In [4]:
import os.path as osp
import numpy as np

both57m_dir = '../data/both57m/'
end_list = []

for data_number in range(3228):
    curr_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
    curr_data = np.load(curr_npz)
    curr_last_frame = curr_data['body_pose'][-1]

    next_npz = osp.join(both57m_dir, str(data_number + 1), 'data.npz')
    next_data = np.load(next_npz)
    next_first_frame = next_data['body_pose'][0]

    diff = np.linalg.norm(curr_last_frame - next_first_frame)

    if diff > 0.1869:
        # print(data_number, np.linalg.norm(curr_last_frame - next_first_frame))
        end_list.append(data_number)

end_list.append(3228)
start_list = [0] + [x + 1 for x in end_list[:-1]]

print(len(start_list), len(end_list))

need_scaling_index = []

data_list = range(3229)
for data_number in data_list:
    if os.path.exists('data/scale_data/%d.pkl' % data_number):
        need_scaling_index.append(data_number)

count = 0
for i in range(len(start_list)):
    if any(x in need_scaling_index for x in range(start_list[i], end_list[i]+1)):
        count += 1
        # print(start_list[i], end_list[i])

print(count)

1383 1383
836


In [10]:
import torch,sys,pickle,os,time
import numpy as np
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv
from scipy.spatial import procrustes

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)

xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
q_init = env.data.qpos.copy()
q_init[18] = -np.pi/2
env.forward(q=q_init)

robot_right_shoulder_pos = env.get_p_joint('robot2/joint1')
robot_right_wrist_pos = env.get_p_joint('robot2/joint6')
robot_left_shoulder_pos = env.get_p_joint('robot1/joint1')

robot_arm_length = np.linalg.norm(robot_right_shoulder_pos-robot_right_wrist_pos)

for data_number in range(0):
    if os.path.exists('scale/%d.npy' % data_number):
        start_time = time.time()
        print('Processing data %d' % data_number)
        xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
        env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)

        with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
            data = pickle.load(f)

        ZERO_ROOT = True
        video = []
        tick = 0
        seq_len = data['length']

        qpos_rig_data = data['qpos']
        ls2le_array = np.zeros((seq_len,3))
        le2lw_array = np.zeros((seq_len,3))
        rs2re_array = np.zeros((seq_len,3))
        re2rw_array = np.zeros((seq_len,3))
        lw_rot_array = np.zeros((seq_len,3,3))
        rw_rot_array = np.zeros((seq_len,3,3))
        lh_array = np.zeros((seq_len,6))
        rh_array = np.zeros((seq_len,6))
        seq_array = np.zeros((2,seq_len,3))

        while tick < seq_len:
            q_rig = qpos_rig_data[tick,:]

            if ZERO_ROOT:
                q_rig[3:7] = 0
            
            env.forward(q=q_rig,INCREASE_TICK=True)
            ls = env.get_p_body('left_shoulder')
            le = env.get_p_body('left_elbow')
            lw = env.get_p_body('left_wrist')
            rs = env.get_p_body('right_shoulder')
            re = env.get_p_body('right_elbow')
            rw = env.get_p_body('right_wrist')

            ls2le_array[tick,:] = np_uv(le-ls)
            le2lw_array[tick,:] = np_uv(lw-le)
            rs2re_array[tick,:] = np_uv(re-rs)
            re2rw_array[tick,:] = np_uv(rw-re)
            lw_rot_array[tick,:] = env.get_R_body('left_wrist')
            rw_rot_array[tick,:] = env.get_R_body('right_wrist')
            tick = tick + 1
         
        env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        BODY_NAMES = ['robot1/end_link','robot2/end_link']
        JOINT_NAMES = ['robot1/joint1','robot1/joint2','robot1/joint3','robot1/joint4','robot1/joint5','robot1/joint6',
                    'robot2/joint1','robot2/joint2','robot2/joint3','robot2/joint4','robot2/joint5','robot2/joint6']

        with open('data/scale_data/%d.pkl' % data_number, 'rb') as f:
            scale_data = pickle.load(f)
        SCALE_LEFT_WRIST = scale_data['SCALE_LEFT']
        SCALE_RIGHT_WRIST = scale_data['SCALE_RIGHT']

        ls = env.get_p_body('robot1/link2')
        le = env.get_p_body('robot1/link4')
        lw = env.get_p_body('robot1/link6')
        rs = env.get_p_body('robot2/link2')
        re = env.get_p_body('robot2/link4')
        rw = env.get_p_body('robot2/link6')

        len_ls2le = np.linalg.norm(le-ls)
        len_le2lw = np.linalg.norm(lw-le)
        len_rs2re = np.linalg.norm(re-rs)
        len_re2rw = np.linalg.norm(rw-re)

        q_env_array = np.zeros((seq_len,env.model.nq))

        for scale_tick in tqdm(range(seq_len), desc="Processing"):
            p_target = {}
            delta_lw = len_ls2le * ls2le_array[scale_tick,:] + len_le2lw * le2lw_array[scale_tick,:]
            if SCALE_LEFT_WRIST:
                if delta_lw[0] > 0: delta_lw[0] /= 2
                delta_lw[0] += 0.2
                if delta_lw[1] > 0: delta_lw[1] /= 2
                delta_lw[2] /= 1.5
            p_target['robot1/end_link'] = ls + delta_lw
            delta_rw = len_rs2re * rs2re_array[scale_tick,:] + len_re2rw * re2rw_array[scale_tick,:]
            if SCALE_RIGHT_WRIST:
                if delta_rw[0] > 0: delta_rw[0] /= 2
                delta_rw[0] += 0.2
                if delta_rw[1] < 0: delta_lw[1] /= 2
                delta_rw[2] /= 1.5
            p_target['robot2/end_link'] = rs + delta_rw
            
            R_target = {}
            R_target['robot1/end_link'] = lw_rot_array[scale_tick,:] @ rpy2r(np.radians([0,0,90]))
            R_target['robot2/end_link'] = rw_rot_array[scale_tick,:] @ rpy2r(np.radians([0,0,-90]))

            idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
            q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
            joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
            joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

            q_body = env.get_qpos_joints(JOINT_NAMES)
            body_diff = 1e3
            prev_q_pos = env.data.qpos.copy()

            while body_diff > 1e-3:
                for ik_tick in range(1):
                    J_list,ik_err_list = [],[]
                    for ik_body_name in BODY_NAMES:
                        ik_p_trgt = p_target[ik_body_name]
                        ik_R_trgt = R_target[ik_body_name]
                        IK_P = True
                        IK_R = True
                        J,ik_err = env.get_ik_ingredients(
                            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                            IK_P=IK_P,IK_R=IK_R)
                        J_list.append(J)
                        ik_err_list.append(ik_err)
                    J_stack      = np.vstack(J_list)
                    ik_err_stack = np.hstack(ik_err_list)
                    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                    q_body = q_body + dq[joint_idxs_jac_body]

                    q_body = np.clip(q_body,q_mins_body,q_maxs_body)
                    env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

                    q_env = env.data.qpos.copy()
                    body_diff = np.linalg.norm(q_env - prev_q_pos)
                    prev_q_pos=q_env.copy()

                curr_time = time.time()
                if curr_time - start_time > 15:
                    break

            q_env_array[tick,:] = q_env
            curr_time = time.time()
            if curr_time - start_time > 15:
                break

            robot_left_wrist_pos = env.get_p_body('robot1/end_link')
            robot_right_wrist_pos = env.get_p_body('robot2/end_link')

            robot_left_wrist_list.append(robot_left_wrist_pos)
            robot_right_wrist_list.append(robot_right_wrist_pos)

        robot_left_wrist_array = np.array(robot_left_wrist_list)
        robot_right_wrist_array = np.array(robot_right_wrist_list)

        smplh_wrist_array = np.load('data/smplh_sequence/%d.npy' % data_number)
        smplh_left_wrist_array = smplh_wrist_array[0,:,:]
        smplh_right_wrist_array = smplh_wrist_array[1,:,:]

        print("Processing data %d" % data_number)
        la, lb, ld = procrustes(smplh_left_wrist_array,robot_left_wrist_array)
        ra, rb, rd = procrustes(smplh_right_wrist_array,robot_right_wrist_array)

        # if ld > 0.3 or rd > 0.3:
        #     scale_data = {
        #         'data_number': data_number,
        #         'left': ld,
        #         'right': rd,
        #         'SCALE_LEFT': True if ld > 0.3 else False,
        #         'SCALE_RIGHT': True if rd > 0.3 else False
        #     }

        #     with open('data/scale_data/%d.pkl' % data_number, 'wb') as f:
        #         pickle.dump(scale_data, f)

        print("Left:", ld)
        print("Right:", rd)

        env.close_viewer()

In [28]:
import torch,sys,pickle,os,time
import numpy as np
sys.path.append('../utils')
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,get_idxs,np_uv
from tqdm import tqdm
from scipy.spatial import procrustes

comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


data_list = range(3229)
for data_number in data_list:
    if os.path.exists('data/scale_data/%d.pkl' % data_number):
        start_time = time.time()
        print('Processing data %d' % data_number)
        xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
        env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=False)

        with open('../data/both57m_rigged/%d.pkl' % data_number, 'rb') as f:
            data = pickle.load(f)

        ZERO_ROOT = True
        video = []
        tick = 0
        seq_len = data['length']

        qpos_rig_data = data['qpos']
        ls2le_array = np.zeros((seq_len,3))
        le2lw_array = np.zeros((seq_len,3))
        rs2re_array = np.zeros((seq_len,3))
        re2rw_array = np.zeros((seq_len,3))
        lw_rot_array = np.zeros((seq_len,3,3))
        rw_rot_array = np.zeros((seq_len,3,3))
        lh_array = np.zeros((seq_len,6))
        rh_array = np.zeros((seq_len,6))
        seq_array = np.zeros((2,seq_len,3))

        while tick < seq_len:
            q_rig = qpos_rig_data[tick,:]

            if ZERO_ROOT:
                q_rig[3:7] = 0
            
            env.forward(q=q_rig,INCREASE_TICK=True)
            ls = env.get_p_body('left_shoulder')
            le = env.get_p_body('left_elbow')
            lw = env.get_p_body('left_wrist')
            rs = env.get_p_body('right_shoulder')
            re = env.get_p_body('right_elbow')
            rw = env.get_p_body('right_wrist')

            ls2le_array[tick,:] = np_uv(le-ls)
            le2lw_array[tick,:] = np_uv(lw-le)
            rs2re_array[tick,:] = np_uv(re-rs)
            re2rw_array[tick,:] = np_uv(rw-re)
            lw_rot_array[tick,:] = env.get_R_body('left_wrist')
            rw_rot_array[tick,:] = env.get_R_body('right_wrist')
            tick = tick + 1

        xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
        env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
        q_init = env.data.qpos.copy()
        q_init[18] = -np.pi/2
        env.forward(q=q_init)

        robot_right_shoulder_pos = env.get_p_joint('robot2/joint1')
        robot_right_wrist_pos = env.get_p_joint('robot2/joint6')
        robot_left_shoulder_pos = env.get_p_joint('robot1/joint1')

        robot_arm_length = np.linalg.norm(robot_right_shoulder_pos-robot_right_wrist_pos)

        env.init_viewer(viewer_title='Dual Arms',viewer_width=800,viewer_height=800,viewer_hide_menus=True)
        env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
        env.reset()

        BODY_NAMES = ['robot1/end_link','robot2/end_link']
        JOINT_NAMES = ['robot1/joint1','robot1/joint2','robot1/joint3','robot1/joint4','robot1/joint5','robot1/joint6',
                    'robot2/joint1','robot2/joint2','robot2/joint3','robot2/joint4','robot2/joint5','robot2/joint6']

        ls = env.get_p_body('robot1/link2')
        le = env.get_p_body('robot1/link4')
        lw = env.get_p_body('robot1/link6')
        rs = env.get_p_body('robot2/link2')
        re = env.get_p_body('robot2/link4')
        rw = env.get_p_body('robot2/link6')

        len_ls2le = np.linalg.norm(le-ls)
        len_le2lw = np.linalg.norm(lw-le)
        len_rs2re = np.linalg.norm(re-rs)
        len_re2rw = np.linalg.norm(rw-re)

        q_env_array = np.zeros((seq_len,env.model.nq))
        with open('data/scale_data/%d.pkl' % data_number, 'rb') as f:
            scale_data = pickle.load(f)
        SCALE_LEFT_WRIST = scale_data['SCALE_LEFT']
        SCALE_RIGHT_WRIST = scale_data['SCALE_RIGHT']

        robot_left_wrist_list = []
        robot_right_wrist_list = []

        for scale_tick in tqdm(range(seq_len), desc="Processing"):
            p_target = {}
            delta_lw = len_ls2le * ls2le_array[scale_tick,:] + len_le2lw * le2lw_array[scale_tick,:]
            if SCALE_LEFT_WRIST:
                if delta_lw[0] > 0: delta_lw[0] /= 2
                delta_lw[0] += 0.2
                if delta_lw[1] > 0: delta_lw[1] /= 2
                delta_lw[2] /= 1.5
            p_target['robot1/end_link'] = ls + delta_lw
            delta_rw = len_rs2re * rs2re_array[scale_tick,:] + len_re2rw * re2rw_array[scale_tick,:]
            if SCALE_RIGHT_WRIST:
                if delta_rw[0] > 0: delta_rw[0] /= 2
                delta_rw[0] += 0.2
                if delta_rw[1] < 0: delta_lw[1] /= 2
                delta_rw[2] /= 1.5
            p_target['robot2/end_link'] = rs + delta_rw
            
            R_target = {}
            R_target['robot1/end_link'] = lw_rot_array[scale_tick,:] @ rpy2r(np.radians([0,0,90]))
            R_target['robot2/end_link'] = rw_rot_array[scale_tick,:] @ rpy2r(np.radians([0,0,-90]))

            idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
            q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
            joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=JOINT_NAMES)
            joint_idxs_jac_body = env.get_idxs_jac(joint_names=JOINT_NAMES)

            q_body = env.get_qpos_joints(JOINT_NAMES)
            body_diff = 1e3
            prev_q_pos = env.data.qpos.copy()

            while body_diff > 1e-3:
                for ik_tick in range(1):
                    J_list,ik_err_list = [],[]
                    for ik_body_name in BODY_NAMES:
                        ik_p_trgt = p_target[ik_body_name]
                        ik_R_trgt = R_target[ik_body_name]
                        IK_P = True
                        IK_R = True
                        J,ik_err = env.get_ik_ingredients(
                            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                            IK_P=IK_P,IK_R=IK_R)
                        J_list.append(J)
                        ik_err_list.append(ik_err)
                    J_stack      = np.vstack(J_list)
                    ik_err_stack = np.hstack(ik_err_list)
                    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                    q_body = q_body + dq[joint_idxs_jac_body]

                    q_body = np.clip(q_body,q_mins_body,q_maxs_body)
                    env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)

                    q_env = env.data.qpos.copy()
                    body_diff = np.linalg.norm(q_env - prev_q_pos)
                    prev_q_pos=q_env.copy()

                # curr_time = time.time()
                # if curr_time - start_time > 15:
                #     break

            robot_left_wrist_pos = env.get_p_body('robot1/end_link')
            robot_right_wrist_pos = env.get_p_body('robot2/end_link')

            robot_left_wrist_list.append(robot_left_wrist_pos)
            robot_right_wrist_list.append(robot_right_wrist_pos)

            q_env_array[scale_tick,:] = q_env
            # curr_time = time.time()
            # if curr_time - start_time > 15:
            #     break

        robot_left_wrist_array = np.array(robot_left_wrist_list)
        robot_right_wrist_array = np.array(robot_right_wrist_list)

        smplh_wrist_array = np.load('data/smplh_sequence/%d.npy' % data_number)
        smplh_left_wrist_array = smplh_wrist_array[0,:,:]
        smplh_right_wrist_array = smplh_wrist_array[1,:,:]

        la, lb, ld = procrustes(smplh_left_wrist_array,robot_left_wrist_array)
        ra, rb, rd = procrustes(smplh_right_wrist_array,robot_right_wrist_array)
        print("Left:", scale_data['left'] - ld)
        print("Right:", scale_data['right'] - rd)

        ours_data = {
            'data_number': data_number,
            'left': ld,
            'right': rd,
            'SCALE_LEFT': scale_data['SCALE_LEFT'],
            'SCALE_RIGHT': scale_data['SCALE_RIGHT'],
            'left_improved': scale_data['left'] - ld,
            'right_improved': scale_data['right'] - rd
        }

        with open('data/ours_data/%d.pkl' % data_number, 'wb') as f:
            pickle.dump(ours_data, f)

        env.close_viewer()


Processing data 0


Processing: 100%|██████████| 150/150 [00:00<00:00, 364.23it/s]


Left: 0.0004021962519741894
Right: 0.4526736156101447
Processing data 2


Processing: 100%|██████████| 150/150 [00:00<00:00, 402.43it/s]


Left: 0.15751879526150214
Right: 0.02734441510395552
Processing data 3


...

Left: 0.07742917540122607
Right: 0.006013994596101413
Processing data 3224


Processing: 100%|██████████| 150/150 [00:01<00:00, 121.58it/s]


Left: 0.00010698519596309686
Right: 0.025600661081137066
Processing data 3225


Processing: 100%|██████████| 150/150 [00:01<00:00, 80.59it/s]

Left: 0.00426116221551065
Right: 0.28337225020264056


In [30]:
data_list = range(3229)
count = 0
improved_distance = 0

for data_number in data_list:
    if os.path.exists('data/ours_data/%d.pkl' % data_number):
        with open('data/ours_data/%d.pkl' % data_number, 'rb') as f:
            data = pickle.load(f)

        if data['SCALE_LEFT']:
            count += 1
            improved_distance += data['left_improved']

        if data['SCALE_RIGHT']:
            count += 1
            improved_distance += data['right_improved']

print("Average improved distance: ", improved_distance / count)


Average improved distance:  0.18805222567355367
